In [45]:
from pathlib import Path
import dis
import marshal
import numpy as np
import types

with open("py_file.pyc", "r+b") as f:
    content = f.read(16)
    code_obj = marshal.load(f)
file_inside = dis.dis(code_obj)
bytecode = bytearray(code_obj.co_code)
bytecode = np.frombuffer(code_obj.co_code, dtype=np.uint8)

BINARY_ADD = dis.opmap["BINARY_ADD"]
BINARY_SUB = dis.opmap["BINARY_SUBTRACT"]

bytecode_new = np.where(bytecode == BINARY_ADD, BINARY_SUB, bytecode).astype(np.uint8)

new_code = types.CodeType(
    code_obj.co_argcount,
    code_obj.co_posonlyargcount if hasattr(code_obj, "co_posonlyargcount") else 0,
    code_obj.co_kwonlyargcount,
    code_obj.co_nlocals,
    code_obj.co_stacksize,
    code_obj.co_flags,
    bytecode_new.tobytes(),  
    code_obj.co_consts,
    code_obj.co_names,
    code_obj.co_varnames,
    code_obj.co_filename,
    code_obj.co_name,
    code_obj.co_firstlineno,
    code_obj.co_lnotab,
    code_obj.co_freevars,
    code_obj.co_cellvars
)

# Disassemble to see result
dis.dis(new_code)

  1           0 LOAD_CONST               0 (5)
              2 STORE_NAME               0 (x)

  2           4 LOAD_NAME                0 (x)
              6 LOAD_NAME                0 (x)
              8 BINARY_ADD
             10 STORE_NAME               1 (y)

  3          12 LOAD_NAME                0 (x)
             14 LOAD_NAME                0 (x)
             16 BINARY_SUBTRACT
             18 STORE_NAME               2 (z)

  4          20 LOAD_NAME                3 (print)
             22 LOAD_CONST               1 ('Hello World!')
             24 CALL_FUNCTION            1
             26 POP_TOP
             28 LOAD_CONST               2 (None)
             30 RETURN_VALUE
  1           0 LOAD_CONST               0 (5)
              2 STORE_NAME               0 (x)

  2           4 LOAD_NAME                0 (x)
              6 LOAD_NAME                0 (x)
              8 BINARY_SUBTRACT
             10 STORE_NAME               1 (y)

  3          12 LOAD_NAME           

In [ ]:
with open("py_file.pyc", "wb") as f:
    f.write(content)                 
    marshal.dump(new_code, f)    


In [ ]:
import py_compile
py_compile.compile('py_file.py', cfile='py_file.pyc')

'py_file.pyc'

In [49]:
with open("py_file.pyc", "r+b") as f:
    content = f.read(16)
    code_obj_changed = marshal.load(f)
dis.dis(code_obj_changed)

  1           0 LOAD_CONST               0 (5)
              2 STORE_NAME               0 (x)

  2           4 LOAD_NAME                0 (x)
              6 LOAD_NAME                0 (x)
              8 BINARY_SUBTRACT
             10 STORE_NAME               1 (y)

  3          12 LOAD_NAME                0 (x)
             14 LOAD_NAME                0 (x)
             16 BINARY_SUBTRACT
             18 STORE_NAME               2 (z)

  4          20 LOAD_NAME                3 (print)
             22 LOAD_CONST               1 ('Hello World!')
             24 CALL_FUNCTION            1
             26 POP_TOP
             28 LOAD_CONST               2 (None)
             30 RETURN_VALUE


In [50]:
import sys
import tracemalloc

# Обычный класс без __slots__
class User:
    def __init__(self, name, surname, age, DOB, email_address, password_hash):
        self.name = name
        self.surname = surname
        self.age = age
        self.DOB = DOB
        self.email_address = email_address
        self.password_hash = password_hash

# Класс с __slots__
class SlottedUser:
    __slots__ = ['name', 'surname', 'age', 'DOB', 'email_address', 'password_hash']
    
    def __init__(self, name, surname, age, DOB, email_address, password_hash):
        self.name = name
        self.surname = surname
        self.age = age
        self.DOB = DOB
        self.email_address = email_address
        self.password_hash = password_hash

def measure_memory(Class, label):
    tracemalloc.start()

    users = [Class("John", "Doe", 30, "1990-01-01", "john@example.com", "hash123") for _ in range(10**6)]

    current, peak = tracemalloc.get_traced_memory()
    print(f"{label}:\n  Current memory usage: {current / 1024 / 1024:.2f} MB\n  Peak memory usage: {peak / 1024 / 1024:.2f} MB\n")

    # Clean up
    del users
    tracemalloc.stop()
   

In [51]:

measure_memory(User, "Without __slots__")
measure_memory(SlottedUser, "With __slots__")

Without __slots__:
  Current memory usage: 191.16 MB
  Peak memory usage: 191.16 MB

With __slots__:
  Current memory usage: 84.36 MB
  Peak memory usage: 84.36 MB



In [ ]:
import http.client

conn = http.client.HTTPSConnection("example.com")
conn.request("GET", "/")
response = conn.getresponse()

print(response.status, response.reason)
print(response.read().decode())

In [ ]:
import requests
import re
import json
import pandas as pd

url = "https://github.com/vectorra/SE_task_1/blob/main/week1/py_file.py"
response = requests.get(url)

html = response.text


match = re.search(r'<script type="application/json" data-target="react-app.embeddedData">(.+?)</script>', response.text, re.DOTALL)
if match:
    script_content = match.group(1)
    data = json.loads(script_content)
    df = pd.DataFrame(data)
    raw_lines = data["payload"]["blob"]["rawLines"]
    print(raw_lines)

code = ""

for i in raw_lines:
    code += i + '\n'
print(code) 

with open("output.py", "w", encoding="utf-8") as f:
    f.write(code)


['x = 5', 'y = x+x', 'z = x-x', 'print("Hello World!")']
x = 5
y = x+x
z = x-x
print("Hello World!")



In [34]:
exec(open("output.py").read())

Hello World!


In [7]:
pip install pandas

  Using cached pandas-2.3.1-cp39-cp39-macosx_11_0_arm64.whl.metadata (91 kB)
  Using cached pytz-2025.2-py2.py3-none-any.whl.metadata (22 kB)
  Using cached tzdata-2025.2-py2.py3-none-any.whl.metadata (1.4 kB)
Using cached pandas-2.3.1-cp39-cp39-macosx_11_0_arm64.whl (10.8 MB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.3/5.3 MB 2.2 MB/s eta 0:00:0000:0100:01
Using cached pytz-2025.2-py2.py3-none-any.whl (509 kB)
Using cached tzdata-2025.2-py2.py3-none-any.whl (347 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4/4 [pandas]2m3/4 [pandas]
Note: you may need to restart the kernel to use updated packages.


In [36]:
import os
import shutil

week1_renames = {
    "task1.py": "task1_main.py",
    "software_task1.ipynb": "task1_notebook.ipynb",
    "py_file.py": "task1_raw.py",
    "py_file.pyc": "task1_raw.pyc",
    "patched_file.pyc": "task1_patched.pyc",
    "output.py": "task1_output.py",
    "output.txt": "task1_output.txt",
}

week2_renames = {
    "Task1_week2.py": "task1_week2.py",
    "ReadMeTask1.txt": "task1_readme.txt",
    "task2.py": "task2_main.py"
}

def organize_files(base_dir, rename_map, target_subfolder):
    os.makedirs(target_subfolder, exist_ok=True)
    for old_name, new_name in rename_map.items():
        src = os.path.join(base_dir, old_name)
        dst = os.path.join(base_dir, target_subfolder, new_name)
        if os.path.exists(src):
            shutil.move(src, dst)
            print(f"Moved and renamed: {src} → {dst}")
        else:
            print(f"Skipped (not found): {src}")

organize_files("../week1", week1_renames, "task1")
organize_files("../week2", {k: v for k, v in week2_renames.items() if "task1" in v}, "task1")
organize_files("../week2", {k: v for k, v in week2_renames.items() if "task2" in v}, "task2")


Moved and renamed: ../week1/task1.py → ../week1/task1/task1_main.py
Moved and renamed: ../week1/software_task1.ipynb → ../week1/task1/task1_notebook.ipynb
Moved and renamed: ../week1/py_file.py → ../week1/task1/task1_raw.py
Moved and renamed: ../week1/py_file.pyc → ../week1/task1/task1_raw.pyc
Moved and renamed: ../week1/patched_file.pyc → ../week1/task1/task1_patched.pyc
Skipped (not found): ../week1/output.py
Skipped (not found): ../week1/output.txt


FileNotFoundError: [Errno 2] No such file or directory: '../week2/task1/task1_week2.py'